In [42]:
import os
from ccdc import conformer
from ccdc import io
from ccdc.descriptors import MolecularDescriptors

In [3]:
mol_path = 'DMOG.mol2'
mol_reader = io.MoleculeReader(mol_path)
mol = mol_reader[0]
mol_reader.close()

In [25]:
conformer_generator = conformer.ConformerGenerator()   # Initialize conformer generator
conformer_generator.settings.max_conformers = 25       # Set max number of conformers
conformers = conformer_generator.generate(mol)         # Run conformer generator

print("""
Conformers generated: %i
Conformer sampling limit reached: %s
Rotamers sampled: %i / %i
""" % (len(conformers), conformers.sampling_limit_reached, conformers.n_rotamers_sampled, conformers.n_rotamers_in_molecule))


Conformers generated: 25
Conformer sampling limit reached: False
Rotamers sampled: 6 / 6



In [44]:
conf_dir = 'DMOG-conformers'
if not os.path.isdir(conf_dir):
    os.makedirs(conf_dir)

molecule_minimiser = conformer.MoleculeMinimiser()                   # Uses Tripos force field
min_conformers = []
for conf_idx, conf in enumerate(conformers):
    score, rmsd = conf.normalised_score, conf.rmsd()                 # Conformer score and RMSD
    min_conf = molecule_minimiser.minimise(conf.molecule)            # Minimise conformer
    min_conformers.append(min_conf)                                  # Add to list
    min_rmsd = round( MolecularDescriptors.rmsd(mol, min_conf), 3)   # Minimized RMSD (!!this method gives different results compared to conf.rmsd())
    
    conf_path = os.path.join(conf_dir, 'dmog-%i.mol2' % (conf_idx + 1))
    with io.MoleculeWriter(conf_path) as molecule_writer:
        molecule_writer.write(min_conf)